# 실시간으로 코인 데이터를 upbit로부터 수집하여 10초 처리 (program_2에서 처리) 를 할 수 있는 만큼 저장하는 프로그램

In [1]:
# !pip install pyupbit
# !pip install websockets

In [1]:
import pyupbit
import uuid
import json
import asyncio
import websockets
import traceback
import nest_asyncio
import pandas as pd
import datetime
nest_asyncio.apply()

In [2]:
 def get_all_ticker():
    """
    데이터 수신할 TICKER 목록
    """
    krw_ticker = pyupbit.get_tickers(fiat="KRW-BTC")
    return krw_ticker

In [3]:
if __name__ == "__main__":   
    print(get_all_ticker())

['KRW-BTC']


In [4]:
data_list = []
i2 = 0

async def connect_socket(lock):
    """
    UPBIT 소켓연결(샘플)
    :return: None
    """
    i2 = 0
    while True:
        try:
            async with websockets.connect('wss://api.upbit.com/websocket/v1', ping_interval=None, ping_timeout=30, max_queue=10000) as websocket:
                subscribe_fmt = [
                    {'ticket':  str(uuid.uuid4())[:1]},
                    {
                        'type': 'ticker',
                        'codes': get_all_ticker(),
                        'isOnlyRealtime': True
                    },
                    {'ticket':  str(uuid.uuid4())[:1]},
                    {
                        'type': 'orderbook',
                        'codes': get_all_ticker(),
                        'isOnlyRealtime': True
                    }
                ]
                subscribe_data = json.dumps(subscribe_fmt)
                await websocket.send(subscribe_data)
 
                while True:
                    try:
                        data = await websocket.recv()
                        data = json.loads(data)
                        data['mytime'] = datetime.datetime.now()
                        
                        data['myid'] = i2
                        i2+=1

                        if 'code' not in data:  # 응답 데이터(딕셔너리)에 code가 없는 경우 제외
                            print("[Data error]", data)
                            continue
 
                        # base_ticker = data['code'].split('-')[0]  # KRW-BTC > KRW(기준통화)
                        ticker = data['code'].split('-')[1]  # KRW-BTC > BTC(시세조회대상 코인)
 
                        for i in range(15):
                            if data['type'] == 'orderbook':
                                data['orderbook_ap_{}'.format(str(i))] = data['orderbook_units'][i]['ask_price']
                                data['orderbook_as_{}'.format(str(i))] = data['orderbook_units'][i]['ask_size']
                                data['orderbook_bp_{}'.format(str(i))] = data['orderbook_units'][i]['bid_price']
                                data['orderbook_bs_{}'.format(str(i))] = data['orderbook_units'][i]['bid_size']
                        
                        if data['type'] == 'orderbook':
                            del data['orderbook_units']
                        
                        # print(ticker, data)

                        async with lock:
                            data_list.append(data)
                            if (data_list[-1].get('mytime') - data_list[0].get('mytime')).total_seconds() > 50:
                                del data_list[0]
                            pd.DataFrame(data_list).to_csv('temp3.csv')
                                                    
                    except (asyncio.TimeoutError, websockets.exceptions.ConnectionClosed):
                        try:
                            pong = await websocket.ping()
                            await asyncio.wait_for(pong, timeout=30)
                        except:
                            await asyncio.sleep(30)
                            break
                    except:
                        traceback.print_exc()
        except:
            await asyncio.sleep(30)
            


In [5]:
async def run():
    
    lock = asyncio.Lock()
    await asyncio.wait([
        asyncio.create_task(connect_socket(lock)),
    ])
    

In [ ]:
if __name__ == "__main__":
    asyncio.run(run())

In [8]:
pd.DataFrame(data_list)

BTC {'type': 'orderbook', 'code': 'KRW-BTC', 'timestamp': 1682936825759, 'total_ask_size': 0.27967468, 'total_bid_size': 5.62135916, 'stream_type': 'REALTIME', 'mytime': datetime.datetime(2023, 5, 1, 19, 27, 5, 799774), 'myid': 50, 'orderbook_ap_0': 38335000.0, 'orderbook_as_0': 0.00514679, 'orderbook_bp_0': 38314000.0, 'orderbook_bs_0': 0.4737, 'orderbook_ap_1': 38337000.0, 'orderbook_as_1': 0.07312434, 'orderbook_bp_1': 38313000.0, 'orderbook_bs_1': 0.48941494, 'orderbook_ap_2': 38339000.0, 'orderbook_as_2': 0.06151556, 'orderbook_bp_2': 38312000.0, 'orderbook_bs_2': 0.09281665, 'orderbook_ap_3': 38340000.0, 'orderbook_as_3': 0.0731404, 'orderbook_bp_3': 38311000.0, 'orderbook_bs_3': 0.48837174, 'orderbook_ap_4': 38341000.0, 'orderbook_as_4': 0.00013041, 'orderbook_bp_4': 38310000.0, 'orderbook_bs_4': 0.28930126, 'orderbook_ap_5': 38342000.0, 'orderbook_as_5': 0.00013041, 'orderbook_bp_5': 38309000.0, 'orderbook_bs_5': 0.005, 'orderbook_ap_6': 38343000.0, 'orderbook_as_6': 0.00013041

,type,code,opening_price,high_price,low_price,trade_price,prev_closing_price,acc_trade_price,change,change_price,...,orderbook_bp_12,orderbook_bs_12,orderbook_ap_13,orderbook_as_13,orderbook_bp_13,orderbook_bs_13,orderbook_ap_14,orderbook_as_14,orderbook_bp_14,orderbook_bs_14
0,ticker,KRW-BTC,39149000.0,39303000.0,38066000.0,38335000.0,39149000.0,1.112829e+11,FALL,814000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,orderbook,KRW-BTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38297000.0,0.033516,38351000.0,0.000261,38296000.0,1.526025,38352000.0,0.400420,38295000.0,0.318976
2,ticker,KRW-BTC,39149000.0,39303000.0,38066000.0,38335000.0,39149000.0,1.112841e+11,FALL,814000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,orderbook,KRW-BTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38298000.0,0.416896,38350000.0,0.017189,38297000.0,0.033516,38351000.0,0.000261,38296000.0,1.526025
4,orderbook,KRW-BTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38297000.0,0.033516,38350000.0,0.017189,38296000.0,1.526025,38351000.0,0.076661,38295000.0,0.318976
5,orderbook,KRW-BTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38297000.0,0.033516,38350000.0,0.017189,38296000.0,1.526025,38351000.0,0.534661,38295000.0,0.318976
6,orderbook,KRW-BTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38297000.0,0.033516,38350000.0,0.017189,38296000.0,1.526025,38351000.0,0.458261,38295000.0,0.318976
7,orderbook,KRW-BTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38297000.0,0.033516,38350000.0,0.017189,38296000.0,1.526025,38351000.0,0.458261,38295000.0,0.318976
8,ticker,KRW-BTC,39149000.0,39303000.0,38066000.0,38312000.0,39149000.0,1.112849e+11,FALL,837000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,orderbook,KRW-BTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38296000.0,1.526025,38349000.0,0.015750,38295000.0,0.318976,38350000.0,0.017189,38294000.0,1.189275


BTC {'type': 'orderbook', 'code': 'KRW-BTC', 'timestamp': 1682936825939, 'total_ask_size': 1.29317468, 'total_bid_size': 5.15026663, 'stream_type': 'REALTIME', 'mytime': datetime.datetime(2023, 5, 1, 19, 27, 6, 2742), 'myid': 51, 'orderbook_ap_0': 38335000.0, 'orderbook_as_0': 0.00514679, 'orderbook_bp_0': 38315000.0, 'orderbook_bs_0': 0.4842, 'orderbook_ap_1': 38337000.0, 'orderbook_as_1': 0.07312434, 'orderbook_bp_1': 38313000.0, 'orderbook_bs_1': 0.00782241, 'orderbook_ap_2': 38339000.0, 'orderbook_as_2': 0.06151556, 'orderbook_bp_2': 38312000.0, 'orderbook_bs_2': 0.09281665, 'orderbook_ap_3': 38340000.0, 'orderbook_as_3': 0.0731404, 'orderbook_bp_3': 38311000.0, 'orderbook_bs_3': 0.48837174, 'orderbook_ap_4': 38341000.0, 'orderbook_as_4': 0.00013041, 'orderbook_bp_4': 38310000.0, 'orderbook_bs_4': 0.28930126, 'orderbook_ap_5': 38342000.0, 'orderbook_as_5': 0.00013041, 'orderbook_bp_5': 38309000.0, 'orderbook_bs_5': 0.005, 'orderbook_ap_6': 38343000.0, 'orderbook_as_6': 0.00013041, 